В данном файле реализуем 2 этапа работы с данными: получение данных и первоначальная подготовка. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
from pathlib import Path
from PIL import Image, ImageDraw
import random
import shutil 
%matplotlib inline

### Этап 1. Получение данных 

В данном проекте будем работать с котировками SBER, за 2020 год, тайм фрейм 1 минута.  
Реализуем простую функцию для скачивания котировок - `get_moex_data`

In [3]:
import requests

def get_moex_data(ticker, start_date, end_date, timeframe):
    url = f"https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/{ticker}/candles.json"
    all_data = []                  # Список для хранения всех данных
    start = 0                      # Начальная позиция для пагинации
    limit = 500                    # Максимальное количество строк за один запрос

    while True:                                                              
        params = {'from': start_date, 'till': end_date, 'interval': timeframe, 'start': start, 'limit': limit}
        response = requests.get(url, params=params)
        data = response.json()
        
        candles = data['candles']['data']
        if not candles:            # Если данных больше нет, выходим из цикла
            break
        
        all_data.extend(candles)   # Добавляем данные в общий список
        start += limit             # Увеличиваем начальную позицию для следующего запроса

    # Создаем DataFrame из всех данных
    columns = data['candles']['columns']
    df = pd.DataFrame(all_data, columns=columns)
    
    return df

Получим и сохраним данные в формате CSV.

In [5]:
# Пример вызова с таймфреймом 1 минута
df = get_moex_data('SBER', '2020-01-01', '2020-12-31', timeframe=1)
df.to_csv("2020_SBER.csv", index=False, sep=';')
print(f"Загружено {len(df)} строк.")

Загружено 169932 строк.


Итак, данные для дальнейшей работы получены, этап 1 на этом завершен. Далее будем работать с сохраненным датасетом - `"2020_SBER.csv"`

### Этап 2. Предобработка данных  
Данный этап, возможно, самый сложный во всем проекте. Можно выделить 3 подэтапа:  
- 2.1. Работа со временем котировок
- 2.2. Добавление дополнительной информации (скользящие средние и т.п.)
- 2.3. Разбиение на классы

**2.1. Работа со временем котировок**  
Изменения ниже нужны для того, чтобы впоследствии более правильно рисовать изображения для новых 15-минутных интервалов.
- Приводим минутное время к 15-минутным интервалам. Время интервала 18:45 - 18:50 отнесем к периоду 18:30.
- Для времени аукциона закрытия 18:45 - надо удалить дубликаты строк с 'Open' == 'High' == 'Low' == 'Close'.

Считываем данные и заводим несколько новых колонок, для более удобной работы.

In [8]:
df = pd.read_csv('2020_SBER.csv', encoding='windows-1251', sep=';')

In [10]:
df['datetime'] = pd.to_datetime(df['begin'])
df['Date'] = df['datetime'].dt.date.astype(str)
df['Time'] = df['datetime'].dt.time.astype(str)
df = df.drop(['value', 'begin', 'end'], axis=1)
df = df.rename(columns={'open': 'Open', 'close': 'Close', 'high': 'High', 'low': 'Low', 'volume': 'Vol'})

In [11]:
# Убираем время 9:59
df = df[df['Time'] != '09:59:00'].reset_index(drop=True)

Приводим минутное время к 15-минутным интервалам. Время интервала 18:45 - 18:50 отнесем к периоду 18:30.

In [12]:
# Преобразуем Time в целочисленный формат hhmmss
df['Time15'] = df['Time'].str.replace(':', '').astype(int)

In [13]:
# Просто костыль для создания списка всех интервалов времени, по типу (100000, 101500) и тд
def time_ranger(a=100000, c=1500, d=5500, t=[]):    
    for i in range(10, 24):
        for _ in range(4):
            p = [a, a + 1500]
            if a % 10000 == 4500:
                p = [a, a + 5500]
            t.append(p)
            if a % 10000 == 4500:
                a += d
            else:
                a += c
    return t
t = time_ranger()
t[34][-1] = 190000
del t[35]
# print(t)

In [14]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [15]:
# приводим мин время к 15 мин интервалам.
for i in t:   
    df.Time15[(df['Time15'] >= i[0]) & (df['Time15'] < i[1])] = i[0]  # Запись в столбец df['Time15']

Для времени аукциона закрытия 18:45 удаляем дубликаты строк. Если ['Open', 'High', 'Low', 'Close'] следующей строки равны ['Open', 'High', 'Low', 'Close'] нашей, то удаляем из датасета дубликаты.

In [16]:
df = df.drop_duplicates(subset=['Open', 'High', 'Low', 'Close'], keep='first')
df.reset_index(drop=True, inplace=True)

**2.2. Добавление дополнительной информации**.  
Скользящие средние, дата-время 15-мин. интервалов, Bollinger Bands. Также колонку 'Y' для идентификации класса.

In [17]:
df['ma_20'] = round(df.Close.rolling(window=20).mean(), 2)
df['ma_40'] = round(df.Close.rolling(window=40).mean(), 2)
df['STD'] = round(df.Close.rolling(window=20).std(), 2)
df['BB_up'] = round(df['ma_20'] + (2 * df['STD']), 2)
df['BB_low'] = round(df['ma_20'] - (2 * df['STD']), 2)
df['dt15'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time15'].astype(str))
df['Y'] = 0

In [18]:
# df.head()

In [19]:
# df.columns

In [20]:
df = df.reindex(columns=['Date', 'Time', 'datetime', 'Time15', 'dt15', 'Open', 'High', 'Low', 
                         'Close', 'Vol', 'ma_20', 'ma_40', 'STD', 'BB_up', 'BB_low', 'Y'])
df = df.drop(['Vol', 'STD'], axis=1)

Поскольку завели скользящюю среднюю с периодом 40, то для первых строк значения будут Nan. Поэтому уберем первый день и начнем датасет с новой даты - `'2020-01-06 10:00:00'`.

In [23]:
df = df[df.datetime >= '2020-01-06 10:00:00']
df = df.reset_index(drop=True)

**Точка сохранения 1 и чтения из нее.**

In [24]:
df.to_csv('temp.csv', index=False, sep=';')

In [25]:
df = pd.read_csv('temp.csv', encoding='windows-1251', sep=';')

**2.3. Разбиение на классы.**

На данном этапе предстоит разметить наши 15-мин интервалы времени (`dt15`) на классы. Для этого создадим функцию `classifier`.  
Суть разметки на классы состоит в том, чтобы отсеять «торговый шум». Функция `classifier` будет принимать датафрейм и коэфф которым будем отсеивать торг шум, например, `coef = 0.25`. А также стоп - коэф, например, `stop_coef = 0.1`.  
Логика классификации такая:  
- Если клоза следующей 15-мин свечи выше текущей и больше `coef`, а abs(лой) след не больше `stop_coef`, то класс 1 - уверенный рост
- Если клоза следующей 15-мин свечи ниже текущей и меньше `-coef`, а хай след не больше `stop_coef`, то класс 3 - уверенное падение
- Остальное, класс 2 - "торговый шум".

In [26]:
df.head(3)

,Date,Time,datetime,Time15,dt15,Open,High,Low,Close,ma_20,ma_40,BB_up,BB_low,Y
0,2020-01-06,10:00:00,2020-01-06 10:00:00,100000,2020-01-06 10:00:00,254.75,254.84,253.85,253.85,254.83,254.83,255.41,254.25,0
1,2020-01-06,10:01:00,2020-01-06 10:01:00,100000,2020-01-06 10:00:00,253.85,254.00,253.03,253.14,254.74,254.80,255.68,253.80,0
2,2020-01-06,10:02:00,2020-01-06 10:02:00,100000,2020-01-06 10:00:00,253.04,253.60,253.04,253.51,254.66,254.78,255.74,253.58,0


In [27]:
def classifier(df, stop_coef, coef):
    
    n_df = np.array(df)
    unique_dt15 = df.dt15.unique()            # Получаем уникальные значения dt15
    results = []    

    for i in range(len(unique_dt15) - 1):
 
        # Фильтруем данные для текущей(df1) и следующей(df2) 15-минутки
        df1 = n_df[n_df[:, 4] == unique_dt15[i]].copy()
        df2 = n_df[n_df[:, 4] == unique_dt15[i + 1]].copy()

        # Вычисления
        cl = df1[:, 8][-1]                    # Последний Close текущей 15-минутки
        cl_next = df2[:, 8][-1]               # Последний Close следующей 15-мин
        high = df2[:, 6].max()                # Максимум следующей 15-минутки
        low = df2[:, 7].min()                 # Минимум следующей 15-минутки
        dh = (high / cl - 1) * 100            # Дельта хай в процентах от клозы
        dl = (low / cl - 1) * 100             # Дельта лой в процентах от клозы
        dcl_next = (cl_next / cl - 1) * 100   # Дельта клоз в процентах    
    
        # Определение Y (класс: 1 - buy, 2 - nothing, 3 - sell)
        if dcl_next >= coef and abs(dl) <= stop_coef:
            df1[:, -1] = 1
        elif dcl_next <= (-coef) and dh <= stop_coef:
            df1[:, -1] = 3
        else:
            df1[:, -1] = 2

        # Добавляем в список результатов
        results.append(df1)

    # Преобразование списка в numpy array перед созданием DataFrame
    results_array = np.vstack(results)
    
    return pd.DataFrame(results_array, columns=df.columns)

Запускаем нашу функцию для присвоения класов.  
Для длины списка уникальных 15-мин (за 1 торг год 2020) `df.dt15.unique()` = 11600, время работы примерно 100 - 120 сек = 2-мин.

In [28]:
# Запускаем нашу функцию для присвоения класов
coef = 0.25
stop_coef = 0.1
df_class = classifier(df, stop_coef, coef)

Глянем сколько получилось объектов каждого класса. Тут будет сильный дисбаланс в пользу класса 2. Примерно 86% 15-мин свечей по логике нашего классификатора это «торговый шум». Логика не идеальна, но приемлема в рамках данного проекта.  
Дисбаланс будет устраняться на следующих этапах.

In [41]:
df_class.groupby('Y')['dt15'].nunique()

Y
1     753
2    9868
3     790
Name: dt15, dtype: int64

Итак, этап предварительной подготовки данных завершен. Сохраним итоги работы.  
**Точка сохранения 2**

In [44]:
df_class.to_csv('df_class.csv', index=False, sep=';')

Далее переходим на следующие этапы работы с данными (см. следующий файл Pic_and_split):  
- Создание изображений в соответствии  с полученной от классификатора разметкой.  
- Разбиение датасета из картинок на тренировочную, валидационную и тестовую выборки.
